In [32]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
import pandas as pd
import numpy as np
import re
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [33]:
df=pd.DataFrame()
df_test=pd.DataFrame()
df=pd.read_csv("dataset_total.csv", encoding="utf-8")
df_test=pd.read_csv("test_set.csv", encoding="utf-8")

In [34]:
all_sentence=df["sentence"].values.tolist()
y_train= df.loc[:1999, "label"].values
y_test= df.loc[2000:, "label"].values

In [26]:
with open("stopword.txt", 'r', encoding='utf-8-sig') as f:
    stop_words= f.read()
    stop_words=stop_words.replace(" ", "")
    stop_words=stop_words.split("\n")

In [35]:
processed_sen=[]
for sentence in all_sentence:
    sentence= re.sub('[১২৩৪৫৬৭৮৯০]', '', sentence)
    sentence= re.sub(' +', ' ', sentence)
    sentence=sentence.split()
    sentence=[w for w in sentence if not w in stop_words]
    processed_sen.append(sentence)

In [36]:
test_x=df_test["documents"].values.tolist()
test_y=df_test["summaries"].values.tolist()
test_x_doc=[]
for document in test_x:
    document=document.split("।")
    test_x_doc.append(document)
    
processed_test_x=[]
for doc in test_x_doc:
    processed_test_doc=[]
    for sentence in doc:
        sentence= re.sub('[১২৩৪৫৬৭৮৯০]', '', sentence)
        sentence= re.sub(' +', ' ', sentence)
        sentence=sentence.split()
        sentence=[w for w in sentence if not w in stop_words]
        processed_test_doc.append(sentence)
    processed_test_x.append(processed_test_doc)

In [37]:
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(processed_sen)
max_length= max([len(s) for s in processed_sen])
vocab_size = len(tokenizer_obj.word_index)+1

In [38]:
x_train=processed_sen[:2000]
x_test=processed_sen[2000:]

x_tranin_tokens= tokenizer_obj.texts_to_sequences(x_train)
x_test_tokens= tokenizer_obj.texts_to_sequences(x_test)

x_train_pad=pad_sequences(x_tranin_tokens, maxlen=max_length, padding="post")
x_test_pad=pad_sequences(x_test_tokens, maxlen=max_length, padding="post")

In [ ]:
##using GRU
E_D=60
model_gru = Sequential()
model_gru.add(Embedding(vocab_size, E_D, input_length=max_length))
model_gru.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
model_gru.add(Dense(64, activation='relu'))
model_gru.add(Dense(1, activation="sigmoid"))

model_gru.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model_gru.summary()
model_gru.fit(x_train_pad, y_train, batch_size=32, epochs=25, validation_data=(x_test_pad, y_test), verbose=2)

In [ ]:
##using LSTM
from keras.layers import Dense, Dropout
model = Sequential()
model.add(Embedding(vocab_size, output_dim=256))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train_pad, y_train, batch_size=16, epochs=25)
score = model.evaluate(x_test_pad, y_test, batch_size=16)

In [41]:
test_token=  tokenizer_obj.texts_to_sequences(processed_test_x[1])
test_pad=pad_sequences(test_token, maxlen=max_length)

In [17]:
out=model.predict(test_pad)